In [1]:
import pandas as pd
import numpy as np
import copy
import tardis

from tardis.io.config_reader import Configuration

/Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
def set_tardis_config_property(tardis_config, key, value):
    keyitems = key.split('.')
    tmp_dict = getattr(tardis_config, keyitems[0])
    for key in keyitems[1:-1]:
        tmp_dict = getattr(tmp_dict, key)
    setattr(tmp_dict, keyitems[-1], value)
    return
        

class tardisGrid:
    
    def __init__(self, configFile, gridFrame):
        try:
            tardis_config = Configuration.from_yaml(configFile)
        except TypeError:
            tardis_config = Configuration.from_config_dict(configFile)
            
        self.config = tardis_config
        self.grid = gridFrame
        
        return
    
    def grid_row_to_config(self, row_index):
        tmp_config = copy.deepcopy(self.config)
        grid_row = self.grid.iloc[row_index]
        for colname, value in zip(self.grid.columns, grid_row.values):
            set_tardis_config_property(tmp_config, colname, value)
        return tmp_config
        
    
    def run_sim_from_grid(self, row_index):
        tardis_config = self.grid_row_to_config(row_index)
        tardis_config.pop('config_dirname')
        sim = tardis.run_tardis(tardis_config)
        return sim
    
    def save_grid(self, filename):
        self.grid.to_csv(filename, index=False)
        return
    
    @classmethod
    def from_axes(cls, configFile, axesdict):
        axes = []
        dim = 1
        for key in axesdict:
            ax = axesdict[key]
            axes.append(ax)
            dim = dim * len(ax)
        axesmesh = np.meshgrid(*axes)
        tmp = np.dstack(axesmesh)
        gridpoints = tmp.reshape((dim, len(axes)), order='F')
        df = pd.DataFrame(data=gridpoints, columns=axesdict.keys())
        return cls(configFile=configFile, gridFrame=df)
    
    

In [3]:
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 'model.abundances.He':np.arange(0,1,0.1),
           'model.abundances.H':np.arange(0,1,0.25)}

In [4]:
grid = tardisGrid.from_axes('test.yml', axesdict)

In [5]:
grid.grid

,model.structure.velocity.start,model.abundances.He,model.abundances.H
0,10000.0,0.0,0.00
1,10000.0,0.1,0.00
2,10000.0,0.2,0.00
3,10000.0,0.3,0.00
4,10000.0,0.4,0.00
...,...,...,...
195,14000.0,0.5,0.75
196,14000.0,0.6,0.75
197,14000.0,0.7,0.75
198,14000.0,0.8,0.75


In [7]:
newconfig = grid.grid_row_to_config(195)

In [11]:
print(newconfig.model.abundances)
print(grid.config.model.abundances)

{'type': 'uniform', 'He': 0.5, 'H': 0.75, 'N': 0.05, 'Fe': 0.015, 'O': 0.0, 'Ni': 0.2505, 'C': 0.008, 'Si': 0.001, 'Ca': 0.0102}
{'type': 'uniform', 'He': 0.6658, 'H': 0.0, 'N': 0.05, 'Fe': 0.015, 'O': 0.0, 'Ni': 0.2505, 'C': 0.008, 'Si': 0.001, 'Ca': 0.0102}


In [12]:
sim = grid.run_sim_from_grid(195)

[tardis.model.base   ][WARNING]  Abundances have not been normalized to 1. - normalizing (base.py:440)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from /Users/marcwilliamson/Research/TARDIS/tardis-refdata/atom_data/kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:87)
[py.warnings         ][WARNING]  /Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
 (warnings.py:110)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:172)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data (base.py:178)
[tardis.plasma.properties.atomic][WARNING]  Zeta_data missing - replaced with 1s. Missing ions: [(14, 15), (20, 21), (26, 27), (28, 29)] (atomic.py:257)
[tardis.plasma.propert

[py.warnings         ][WARNING]  /Users/marcwilliamson/src/dev/tardis/tardis/montecarlo/montecarlo_numba/base.py:178: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../montecarlo/montecarlo_numba/r_packet.py", line 233:
@njit(**njit_dict)
def get_doppler_factor(r, mu, time_explosion):
^

  r_packet, numba_model, numba_plasma, estimators, vpacket_collection
 (warnings.py:110)
[py.warnings         ][WARNING]  /Users/marcwilliamson/src/dev/tardis/tardis/montecarlo/montecarlo_numba/base.py:178: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics 

[py.warnings         ][WARNING]  /Users/marcwilliamson/src/dev/tardis/tardis/montecarlo/montecarlo_numba/base.py:178: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../montecarlo/montecarlo_numba/r_packet.py", line 249:
@njit(**njit_dict)
def get_doppler_factor_full_relativity(mu, beta):
^

  r_packet, numba_model, numba_plasma, estimators, vpacket_collection
 (warnings.py:110)
[py.warnings         ][WARNING]  /Users/marcwilliamson/src/dev/tardis/tardis/montecarlo/montecarlo_numba/base.py:178: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnosti

[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad   next_t_rad         w    next_w
	Shell                                              
	0      6996.499329  7349.490364  0.427390  0.456755
	5      6961.684230  7421.633545  0.276535  0.253515
	10     6927.213900  7308.543382  0.211205  0.181286
	15     6893.083244  7017.562145  0.169661  0.145029

 (base.py:436)
[tardis.simulation.base][INFO   ]  t_inner 7000.000 K -- next t_inner 6463.406 K (base.py:439)
[tardis.simulation.base][INFO   ]  Starting iteration 2/20 (base.py:326)
[py.warnings         ][WARNING]  /Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 5.47283e+41 erg / s Luminosity absorbed = 6.13378e+41 e

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 5.73581e+41 erg / s Luminosity absorbed = 6.57023e+41 erg / s Luminosity requested = 5.67900e+41 erg / s (base.py:450)
[tardis.simulation.base][INFO   ]  Iteration converged 1/4 consecutive times. (base.py:229)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad   next_t_rad         w    next_w
	Shell                                              
	0      7267.632743  7287.134698  0.474849  0.477064
	5      7711.869677  7751.994159  0.199750  0.198575
	10     7608.899235  7630.972637  0.119465  0.118624
	15     7072.255664  7100.240486  0.089825  0.089267

 (base.py:436)
[tardis.simulation.base][INFO   ]  t_inner 6551.160 K -- next t_inner 6534.899 K (base.py:439)
[tardis.simulation.base][INFO   ]  Starting iteration 9/20 (base.py:326)
[py.warnings         ][WARNING]  /Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/site-packages/astr

[tardis.simulation.base][INFO   ]  t_inner 6543.830 K -- next t_inner 6534.336 K (base.py:439)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:326)
[py.warnings         ][WARNING]  /Users/marcwilliamson/miniconda3/envs/numbatardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 5.72245e+41 erg / s Luminosity absorbed = 6.46690e+41 erg / s Luminosity requested = 5.67900e+41 erg / s (base.py:450)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad   next_t_rad         w    next_w
	Shell                                              
	0      7279.506950  7280.808946  0.477163  0.478163
	5      7738.982356  7738.510409  0.201584  0.200897
	10     7610.088876  7636.499177  0.120119  0.118251
	15 

In [5]:
grid.config

{'tardis_config_version': 'v1.0',
 'supernova': {'luminosity_requested': <Quantity 5.679e+41 erg / s>,
  'luminosity_wavelength_start': <Quantity 3481.82000178 Angstrom>,
  'luminosity_wavelength_end': <Quantity 9947.78776065 Angstrom>,
  'time_explosion': <Quantity 10. d>},
 'atom_data': '/Users/marcwilliamson/Research/TARDIS/tardis-refdata/atom_data/kurucz_cd23_chianti_H_He.h5',
 'model': {'structure': {'type': 'specific',
   'velocity': {'start': <Quantity 10000. km / s>,
    'stop': <Quantity 20000. km / s>,
    'num': 20},
   'density': {'type': 'branch85_w7',
    'w7_time_0': <Quantity 0.00023148 d>,
    'w7_rho_0': <Quantity 3.e+29 g / cm3>,
    'w7_v_0': <Quantity 1. km / s>}},
  'abundances': {'type': 'uniform',
   'He': 0.6658,
   'H': 0.0,
   'N': 0.05,
   'Fe': 0.015,
   'O': 0.0,
   'Ni': 0.2505,
   'C': 0.008,
   'Si': 0.001,
   'Ca': 0.0102}},
 'plasma': {'initial_t_inner': <Quantity 7000. K>,
  'disable_electron_scattering': False,
  'ionization': 'nebular',
  'excitati

In [7]:
grid.config.plasma.initial_t_inner

<Quantity 7000. K>